In [ ]:
!pip install scikit-optimize
!pip install shap

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot
import os

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LassoCV
from sklearn.model_selection import RepeatedKFold
from skopt import BayesSearchCV
from sklearn.metrics import (accuracy_score, auc, classification_report, confusion_matrix, f1_score, matthews_corrcoef, precision_recall_curve, precision_score, recall_score, roc_auc_score, roc_curve)
from sklearn.utils import resample

import scipy.stats as stats

from imblearn.over_sampling import SMOTE

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

import shap

import cv2
from google.colab.patches import cv2_imshow
from PIL import Image

pd.set_option('display.max_rows', None)

# Data preprocessing

In [ ]:
#Open csv file.

data = pd.read_csv("/content/drive/MyDrive/HCMP-ML/results.csv", index_col=0)
data.head()

In [ ]:
#Define predictor variables (x) and outcome of interest (y).

x = data.drop(['Outcome'], axis = 1)
y = data['Outcome']

In [ ]:
#Normalize data.

x[x.columns] = MinMaxScaler().fit_transform(x[x.columns])

In [ ]:
#Check data shapes.

print(y.shape)
print(x.shape)

In [ ]:
def bootstrap_ci(metric_list, n_bootstraps=1000, alpha=0.05):
    bootstrapped_metrics = []
    for _ in range(n_bootstraps):
        bootstrapped_metric = np.mean(resample(metric_list, replace=True, n_samples=len(metric_list)))
        bootstrapped_metrics.append(bootstrapped_metric)

    lower_bound = np.percentile(bootstrapped_metrics, alpha / 2 * 100)
    upper_bound = np.percentile(bootstrapped_metrics, (1 - alpha / 2) * 100)
    return lower_bound, upper_bound

# Feature selection

In [ ]:
# Define the range of alpha values to search
alphas = np.logspace(-10, 1, 100)

# Perform Lasso feature selection with the specified range of alphas
lasso = LassoCV(alphas=alphas, max_iter=100, cv=5, random_state=42)
lasso.fit(x, y)

# Get the indices of the non-zero features
selected_features_idx = np.nonzero(lasso.coef_)[0]

# Get the selected feature names
selected_features = x.columns[selected_features_idx]
print(selected_features)

# Select the non-zero features for all datasets
x = x[selected_features]

# Models

## XGBoost

In [ ]:
# Initialize 5-fold cross-validator.
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)

# Set up the parameter search space for hyperparameter tuning.
search_space = {
    'n_estimators': (100, 500),
    'max_depth': (3, 9),
    'learning_rate': (0.01, 0.2, 'uniform'),
    'subsample': (0.5, 1, 'uniform'),
    'colsample_bytree': (0.5, 1, 'uniform'),
    'gamma': (0, 0.3, 'uniform'),
    'min_child_weight': (1, 5)
}

# Initialize XGBClassifier.
xgb = XGBClassifier(random_state=42)

# Initialize BayesSearchCV with the XGBClassifier, the search space, and k-fold CV.
bayes_search = BayesSearchCV(estimator=xgb, search_spaces=search_space, n_iter=20, cv=cv, scoring='roc_auc', n_jobs=-1, verbose=1, random_state=42)

# Fit the BayesSearchCV on the data.
bayes_search.fit(x.values, y.values)

# Get the best parameters from the BayesSearchCV.
best_params = bayes_search.best_params_

In [ ]:
# Create empty lists to store metrics for each fold.

xgb_precision_list = []
xgb_recall_list = []
xgb_f1_list = []
xgb_acc_list = []
xgb_mcc_list = []
xgb_auroc_list = []
xgb_auprc_list = []
xgb_tpr_list = []
xgb_prc_list = []

xgb_base_fpr = np.linspace(0, 1, 101)

# Initialize SMOTE.

smote = SMOTE(random_state=42)

# Perform cross-validation with Repeated k-fold CV using the best parameters.

for train_index, test_index in cv.split(x, y):
    X_train_fold, X_test_fold = x.iloc[train_index], x.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

    # Apply SMOTE to the training data.

    X_train_fold_resampled, y_train_fold_resampled = smote.fit_resample(X_train_fold, y_train_fold)

    xgb = XGBClassifier(**best_params, random_state=42)
    xgb.fit(X_train_fold_resampled.values, y_train_fold_resampled.values)

    preds_xgb = xgb.predict(X_test_fold.values)
    probs_xgb = xgb.predict_proba(X_test_fold.values)
    probs_xgb = probs_xgb[:, 1]

    # Calculate performance metrics.

    xgb_precision_list.append(precision_score(y_test_fold, preds_xgb))
    xgb_recall_list.append(recall_score(y_test_fold, preds_xgb))
    xgb_f1_list.append(f1_score(y_test_fold, preds_xgb))
    xgb_acc_list.append(accuracy_score(y_test_fold, preds_xgb))
    xgb_mcc_list.append(matthews_corrcoef(y_test_fold, preds_xgb))
    xgb_auroc_list.append(roc_auc_score(y_test_fold, probs_xgb))
    xgb_prc_p, xgb_prc_r, _ = precision_recall_curve(y_test_fold, probs_xgb)
    xgb_auprc_list.append(auc(xgb_prc_r, xgb_prc_p))

    # Calculate ROC and PR curves.

    fpr, tpr, _ = roc_curve(y_test_fold, probs_xgb)
    tpr = np.interp(xgb_base_fpr, fpr, tpr)
    tpr[0] = 0.0
    xgb_tpr_list.append(tpr)

    precision, recall, _ = precision_recall_curve(y_test_fold, probs_xgb)
    prc = np.interp(xgb_base_fpr, recall[::-1], precision[::-1])
    xgb_prc_list.append(prc)

In [ ]:
# Calculate the mean for each metric.

xgb_precision_mean = np.mean(xgb_precision_list)
xgb_recall_mean = np.mean(xgb_recall_list)
xgb_f1_mean = np.mean(xgb_f1_list)
xgb_acc_mean = np.mean(xgb_acc_list)
xgb_mcc_mean = np.mean(xgb_mcc_list)
xgb_auroc_mean = np.mean(xgb_auroc_list)
xgb_auprc_mean = np.mean(xgb_auprc_list)

In [ ]:
# Calculate the confidence intervals for each metric.

metrics = {
    'Precision': xgb_precision_list,
    'Recall': xgb_recall_list,
    'F1 Score': xgb_f1_list,
    'Accuracy': xgb_acc_list,
    'MCC': xgb_mcc_list,
    'AUROC': xgb_auroc_list,
    'AUPRC': xgb_auprc_list
}

result_strings = {}

for metric_name, metric_list in metrics.items():
    mean = round(np.mean(metric_list), 3)
    lower_ci, upper_ci = bootstrap_ci(metric_list)
    result_str = f"{metric_name}: {mean} ({lower_ci:.3f}, {upper_ci:.3f})"
    result_strings[metric_name] = result_str
    print(result_str)

xgb_precision_str = result_strings['Precision']
xgb_recall_str = result_strings['Recall']
xgb_f1_str = result_strings['F1 Score']
xgb_acc_str = result_strings['Accuracy']
xgb_mcc_str = result_strings['MCC']
xgb_auroc_str = result_strings['AUROC']
xgb_auprc_str = result_strings['AUPRC']

In [ ]:
# Calculate ROC and PR curves.

xgb_tpr_list = np.array(xgb_tpr_list)
xgb_mean_tprs = xgb_tpr_list.mean(axis=0)
xgb_std_tprs = xgb_tpr_list.std(axis=0)

xgb_prc_list = np.array(xgb_prc_list)
xgb_mean_prcs = xgb_prc_list.mean(axis=0)
xgb_std_prcs = xgb_prc_list.std(axis=0)

## LightGBM

In [ ]:
# Initialize 5-fold cross-validator.
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)

# Set up the parameter search space for hyperparameter tuning.
search_space = {
    'n_estimators': (100, 500),
    'max_depth': (3, 9),
    'learning_rate': (0.01, 0.2, 'uniform'),
    'subsample': (0.5, 1, 'uniform'),
    'colsample_bytree': (0.5, 1, 'uniform'),
    'min_split_gain': (0, 0.3, 'uniform'),
    'min_child_weight': (1, 5),
    'num_leaves': (31, 64),
}

# Initialize LGBMClassifier.
lgbm = LGBMClassifier(random_state=42)

# Initialize BayesSearchCV with the LGBMClassifier, the search space, and k-fold CV.
bayes_search = BayesSearchCV(estimator=lgbm, search_spaces=search_space, n_iter=20, cv=cv, scoring='roc_auc', n_jobs=-1, verbose=1, random_state=42)

# Fit the BayesSearchCV on the data.
bayes_search.fit(x.values, y.values)

# Get the best parameters from the BayesSearchCV.
best_params = bayes_search.best_params_

In [ ]:
# Create empty lists to store metrics for each fold.

lgb_precision_list = []
lgb_recall_list = []
lgb_f1_list = []
lgb_acc_list = []
lgb_mcc_list = []
lgb_auroc_list = []
lgb_auprc_list = []
lgb_tpr_list = []
lgb_prc_list = []

lgb_base_fpr = np.linspace(0, 1, 101)

# Initialize SMOTE.

smote = SMOTE(random_state=42)

# Perform cross-validation with Repeated k-fold CV using the best parameters.

for train_index, test_index in cv.split(x, y):
    X_train_fold, X_test_fold = x.iloc[train_index], x.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

    # Apply SMOTE to the training data.

    X_train_fold_resampled, y_train_fold_resampled = smote.fit_resample(X_train_fold, y_train_fold)

    lgb = LGBMClassifier(**best_params, random_state=42)
    lgb.fit(X_train_fold_resampled, y_train_fold_resampled)

    preds_lgb = lgb.predict(X_test_fold)
    probs_lgb = lgb.predict_proba(X_test_fold)
    probs_lgb = probs_lgb[:, 1]

    # Calculate performance metrics.

    lgb_precision_list.append(precision_score(y_test_fold, preds_lgb))
    lgb_recall_list.append(recall_score(y_test_fold, preds_lgb))
    lgb_f1_list.append(f1_score(y_test_fold, preds_lgb))
    lgb_acc_list.append(accuracy_score(y_test_fold, preds_lgb))
    lgb_mcc_list.append(matthews_corrcoef(y_test_fold, preds_lgb))
    lgb_auroc_list.append(roc_auc_score(y_test_fold, probs_lgb))
    lgb_prc_p, lgb_prc_r, _ = precision_recall_curve(y_test_fold, probs_lgb)
    lgb_auprc_list.append(auc(lgb_prc_r, lgb_prc_p))

    # Calculate ROC and PR curves.

    fpr, tpr, _ = roc_curve(y_test_fold, probs_lgb)
    tpr = np.interp(lgb_base_fpr, fpr, tpr)
    tpr[0] = 0.0
    lgb_tpr_list.append(tpr)

    precision, recall, _ = precision_recall_curve(y_test_fold, probs_lgb)
    prc = np.interp(lgb_base_fpr, recall[::-1], precision[::-1])
    lgb_prc_list.append(prc)

In [ ]:
# Calculate the mean for each metric.

lgb_precision_mean = np.mean(lgb_precision_list)
lgb_recall_mean = np.mean(lgb_recall_list)
lgb_f1_mean = np.mean(lgb_f1_list)
lgb_acc_mean = np.mean(lgb_acc_list)
lgb_mcc_mean = np.mean(lgb_mcc_list)
lgb_auroc_mean = np.mean(lgb_auroc_list)
lgb_auprc_mean = np.mean(lgb_auprc_list)

In [ ]:
# Calculate the confidence intervals for each metric.

metrics = {
    'Precision': lgb_precision_list,
    'Recall': lgb_recall_list,
    'F1 Score': lgb_f1_list,
    'Accuracy': lgb_acc_list,
    'MCC': lgb_mcc_list,
    'AUROC': lgb_auroc_list,
    'AUPRC': lgb_auprc_list
}

result_strings = {}

for metric_name, metric_list in metrics.items():
    mean = round(np.mean(metric_list), 3)
    lower_ci, upper_ci = bootstrap_ci(metric_list)
    result_str = f"{metric_name}: {mean} ({lower_ci:.3f}, {upper_ci:.3f})"
    result_strings[metric_name] = result_str
    print(result_str)

lgb_precision_str = result_strings['Precision']
lgb_recall_str = result_strings['Recall']
lgb_f1_str = result_strings['F1 Score']
lgb_acc_str = result_strings['Accuracy']
lgb_mcc_str = result_strings['MCC']
lgb_auroc_str = result_strings['AUROC']
lgb_auprc_str = result_strings['AUPRC']

In [ ]:
# Calculate ROC and PR curves.

lgb_tpr_list = np.array(lgb_tpr_list)
lgb_mean_tprs = lgb_tpr_list.mean(axis=0)
lgb_std_tprs = lgb_tpr_list.std(axis=0)

lgb_prc_list = np.array(lgb_prc_list)
lgb_mean_prcs = lgb_prc_list.mean(axis=0)
lgb_std_prcs = lgb_prc_list.std(axis=0)

## Support Vector Machine

In [ ]:
# Initialize 5-fold cross-validator.
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)

# Set up the parameter search space for hyperparameter tuning.
search_space = {
    'C': (1e-6, 1e+6, 'log-uniform'),
    'kernel': ['linear', 'poly', 'rbf'],
    'degree': (2, 5),
    'coef0': (0, 5),
    'shrinking': [True, False],
    'probability': [True],
    'gamma': (1e-6, 1e+1, 'log-uniform'),
}

# Initialize SVC.
svm = SVC(random_state=42)

# Initialize BayesSearchCV with the SVC, the search space, and k-fold CV.
bayes_search = BayesSearchCV(estimator=svm, search_spaces=search_space, n_iter=20, cv=cv, scoring='roc_auc', n_jobs=-1, verbose=1, random_state=42)

# Fit the BayesSearchCV on the data.
bayes_search.fit(x.values, y.values)

# Get the best parameters from the BayesSearchCV.
best_params = bayes_search.best_params_


In [ ]:
# Create empty lists to store metrics for each fold.

svm_precision_list = []
svm_recall_list = []
svm_f1_list = []
svm_acc_list = []
svm_mcc_list = []
svm_auroc_list = []
svm_auprc_list = []
svm_tpr_list = []
svm_prc_list = []

svm_base_fpr = np.linspace(0, 1, 101)

# Initialize SMOTE.

smote = SMOTE(random_state=42)

# Perform cross-validation with Repeated k-fold CV using the best parameters.

for train_index, test_index in cv.split(x, y):
    X_train_fold, X_test_fold = x.iloc[train_index], x.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

    # Apply SMOTE to the training data.

    X_train_fold_resampled, y_train_fold_resampled = smote.fit_resample(X_train_fold, y_train_fold)

    svm = SVC(**best_params, random_state=42)
    svm.fit(X_train_fold_resampled, y_train_fold_resampled)

    preds_svm = svm.predict(X_test_fold)
    probs_svm = svm.predict_proba(X_test_fold)
    probs_svm = probs_svm[:, 1]

    # Calculate performance metrics.

    svm_precision_list.append(precision_score(y_test_fold, preds_svm))
    svm_recall_list.append(recall_score(y_test_fold, preds_svm))
    svm_f1_list.append(f1_score(y_test_fold, preds_svm))
    svm_acc_list.append(accuracy_score(y_test_fold, preds_svm))
    svm_mcc_list.append(matthews_corrcoef(y_test_fold, preds_svm))
    svm_auroc_list.append(roc_auc_score(y_test_fold, probs_svm))
    svm_prc_p, svm_prc_r, _ = precision_recall_curve(y_test_fold, probs_svm)
    svm_auprc_list.append(auc(svm_prc_r, svm_prc_p))

    # Calculate ROC and PR curves.

    fpr, tpr, _ = roc_curve(y_test_fold, probs_svm)
    tpr = np.interp(svm_base_fpr, fpr, tpr)
    tpr[0] = 0.0
    svm_tpr_list.append(tpr)

    precision, recall, _ = precision_recall_curve(y_test_fold, probs_svm)
    prc = np.interp(svm_base_fpr, recall[::-1], precision[::-1])
    svm_prc_list.append(prc)

In [ ]:
# Calculate the mean for each metric.

svm_precision_mean = np.mean(svm_precision_list)
svm_recall_mean = np.mean(svm_recall_list)
svm_f1_mean = np.mean(svm_f1_list)
svm_acc_mean = np.mean(svm_acc_list)
svm_mcc_mean = np.mean(svm_mcc_list)
svm_auroc_mean = np.mean(svm_auroc_list)
svm_auprc_mean = np.mean(svm_auprc_list)

In [ ]:
# Calculate the confidence intervals for each metric.

metrics = {
    'Precision': svm_precision_list,
    'Recall': svm_recall_list,
    'F1 Score': svm_f1_list,
    'Accuracy': svm_acc_list,
    'MCC': svm_mcc_list,
    'AUROC': svm_auroc_list,
    'AUPRC': svm_auprc_list
}

result_strings = {}

for metric_name, metric_list in metrics.items():
    mean = round(np.mean(metric_list), 3)
    lower_ci, upper_ci = bootstrap_ci(metric_list)
    result_str = f"{metric_name}: {mean} ({lower_ci:.3f}, {upper_ci:.3f})"
    result_strings[metric_name] = result_str
    print(result_str)

svm_precision_str = result_strings['Precision']
svm_recall_str = result_strings['Recall']
svm_f1_str = result_strings['F1 Score']
svm_acc_str = result_strings['Accuracy']
svm_mcc_str = result_strings['MCC']
svm_auroc_str = result_strings['AUROC']
svm_auprc_str = result_strings['AUPRC']

In [ ]:
# Calculate ROC and PR curves.

svm_tpr_list = np.array(svm_tpr_list)
svm_mean_tprs = svm_tpr_list.mean(axis=0)
svm_std_tprs = svm_tpr_list.std(axis=0)

svm_prc_list = np.array(svm_prc_list)
svm_mean_prcs = svm_prc_list.mean(axis=0)
svm_std_prcs = svm_prc_list.std(axis=0)

## Random Forest

In [ ]:
# Initialize 5-fold cross-validator.
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)

# Set up the parameter search space for hyperparameter tuning.
search_space = {
    'n_estimators': (10, 500),
    'criterion': ['gini', 'entropy'],
    'max_depth': (3, 50),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 10),
    'max_features': ['auto', 'sqrt', 'log2', None],
    'bootstrap': [True, False],
}

# Initialize RandomForestClassifier.
rf = RandomForestClassifier(random_state=42)

# Initialize BayesSearchCV with the RandomForestClassifier, the search space, and k-fold CV.
bayes_search = BayesSearchCV(estimator=rf, search_spaces=search_space, n_iter=20, cv=cv, scoring='roc_auc', n_jobs=-1, verbose=1, random_state=42)

# Fit the BayesSearchCV on the data.
bayes_search.fit(x.values, y.values)

# Get the best parameters from the BayesSearchCV.
best_params = bayes_search.best_params_


In [ ]:
# Create empty lists to store metrics for each fold.

rf_precision_list = []
rf_recall_list = []
rf_f1_list = []
rf_acc_list = []
rf_mcc_list = []
rf_auroc_list = []
rf_auprc_list = []
rf_tpr_list = []
rf_prc_list = []

rf_base_fpr = np.linspace(0, 1, 101)

# Initialize SMOTE.

smote = SMOTE(random_state=42)

# Perform cross-validation with Repeated k-fold CV using the best parameters.

for train_index, test_index in cv.split(x, y):
    X_train_fold, X_test_fold = x.iloc[train_index], x.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

    # Apply SMOTE to the training data.

    X_train_fold_resampled, y_train_fold_resampled = smote.fit_resample(X_train_fold, y_train_fold)

    rf = RandomForestClassifier(**best_params, random_state=42)
    rf.fit(X_train_fold_resampled, y_train_fold_resampled)

    preds_rf = rf.predict(X_test_fold)
    probs_rf = rf.predict_proba(X_test_fold)
    probs_rf = probs_rf[:, 1]

    # Calculate performance metrics.

    rf_precision_list.append(precision_score(y_test_fold, preds_rf))
    rf_recall_list.append(recall_score(y_test_fold, preds_rf))
    rf_f1_list.append(f1_score(y_test_fold, preds_rf))
    rf_acc_list.append(accuracy_score(y_test_fold, preds_rf))
    rf_mcc_list.append(matthews_corrcoef(y_test_fold, preds_rf))
    rf_auroc_list.append(roc_auc_score(y_test_fold, probs_rf))
    rf_prc_p, rf_prc_r, _ = precision_recall_curve(y_test_fold, probs_rf)
    rf_auprc_list.append(auc(rf_prc_r, rf_prc_p))

    # Calculate ROC and PR curves.

    fpr, tpr, _ = roc_curve(y_test_fold, probs_rf)
    tpr = np.interp(rf_base_fpr, fpr, tpr)
    tpr[0] = 0.0
    rf_tpr_list.append(tpr)

    precision, recall, _ = precision_recall_curve(y_test_fold, probs_rf)
    prc = np.interp(rf_base_fpr, recall[::-1], precision[::-1])
    rf_prc_list.append(prc)

In [ ]:
# Calculate the mean for each metric.

rf_precision_mean = np.mean(rf_precision_list)
rf_recall_mean = np.mean(rf_recall_list)
rf_f1_mean = np.mean(rf_f1_list)
rf_acc_mean = np.mean(rf_acc_list)
rf_mcc_mean = np.mean(rf_mcc_list)
rf_auroc_mean = np.mean(rf_auroc_list)
rf_auprc_mean = np.mean(rf_auprc_list)

In [ ]:
# Calculate the confidence intervals for each metric.

metrics = {
    'Precision': rf_precision_list,
    'Recall': rf_recall_list,
    'F1 Score': rf_f1_list,
    'Accuracy': rf_acc_list,
    'MCC': rf_mcc_list,
    'AUROC': rf_auroc_list,
    'AUPRC': rf_auprc_list
}

result_strings = {}

for metric_name, metric_list in metrics.items():
    mean = round(np.mean(metric_list), 3)
    lower_ci, upper_ci = bootstrap_ci(metric_list)
    result_str = f"{metric_name}: {mean} ({lower_ci:.3f}, {upper_ci:.3f})"
    result_strings[metric_name] = result_str
    print(result_str)

rf_precision_str = result_strings['Precision']
rf_recall_str = result_strings['Recall']
rf_f1_str = result_strings['F1 Score']
rf_acc_str = result_strings['Accuracy']
rf_mcc_str = result_strings['MCC']
rf_auroc_str = result_strings['AUROC']
rf_auprc_str = result_strings['AUPRC']

In [ ]:
# Calculate ROC and PR curves.

rf_tpr_list = np.array(rf_tpr_list)
rf_mean_tprs = rf_tpr_list.mean(axis=0)
rf_std_tprs = rf_tpr_list.std(axis=0)

rf_prc_list = np.array(rf_prc_list)
rf_mean_prcs = rf_prc_list.mean(axis=0)
rf_std_prcs = rf_prc_list.std(axis=0)

## kNN

In [ ]:
# Initialize 5-fold cross-validator.
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)

# Set up the parameter search space for hyperparameter tuning.
search_space = {
    'n_neighbors': (1, 30),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': (10, 50),
    'p': (1, 2),
}

# Initialize KNeighborsClassifier.
knn = KNeighborsClassifier()

# Initialize BayesSearchCV with the KNeighborsClassifier, the search space, and k-fold CV.
bayes_search = BayesSearchCV(estimator=knn, search_spaces=search_space, n_iter=20, cv=cv, scoring='roc_auc', n_jobs=-1, verbose=1, random_state=42)

# Fit the BayesSearchCV on the data.
bayes_search.fit(x.values, y.values)

# Get the best parameters from the BayesSearchCV.
best_params = bayes_search.best_params_

In [ ]:
# Create empty lists to store metrics for each fold.

knn_precision_list = []
knn_recall_list = []
knn_f1_list = []
knn_acc_list = []
knn_mcc_list = []
knn_auroc_list = []
knn_auprc_list = []
knn_tpr_list = []
knn_prc_list = []

knn_base_fpr = np.linspace(0, 1, 101)

# Initialize SMOTE.

smote = SMOTE(random_state=42)

# Peknnorm cross-validation with Repeated k-fold CV using the best parameters.

for train_index, test_index in cv.split(x, y):
    X_train_fold, X_test_fold = x.iloc[train_index], x.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

    # Apply SMOTE to the training data.

    X_train_fold_resampled, y_train_fold_resampled = smote.fit_resample(X_train_fold, y_train_fold)

    knn = KNeighborsClassifier(**best_params)
    knn.fit(X_train_fold_resampled, y_train_fold_resampled)

    preds_knn = knn.predict(X_test_fold)
    probs_knn = knn.predict_proba(X_test_fold)
    probs_knn = probs_knn[:, 1]

    # Calculate peknnormance metrics.

    knn_precision_list.append(precision_score(y_test_fold, preds_knn))
    knn_recall_list.append(recall_score(y_test_fold, preds_knn))
    knn_f1_list.append(f1_score(y_test_fold, preds_knn))
    knn_acc_list.append(accuracy_score(y_test_fold, preds_knn))
    knn_mcc_list.append(matthews_corrcoef(y_test_fold, preds_knn))
    knn_auroc_list.append(roc_auc_score(y_test_fold, probs_knn))
    knn_prc_p, knn_prc_r, _ = precision_recall_curve(y_test_fold, probs_knn)
    knn_auprc_list.append(auc(knn_prc_r, knn_prc_p))

    # Calculate ROC and PR curves.

    fpr, tpr, _ = roc_curve(y_test_fold, probs_knn)
    tpr = np.interp(knn_base_fpr, fpr, tpr)
    tpr[0] = 0.0
    knn_tpr_list.append(tpr)

    precision, recall, _ = precision_recall_curve(y_test_fold, probs_knn)
    prc = np.interp(knn_base_fpr, recall[::-1], precision[::-1])
    knn_prc_list.append(prc)

In [ ]:
# Calculate the mean for each metric.

knn_precision_mean = np.mean(knn_precision_list)
knn_recall_mean = np.mean(knn_recall_list)
knn_f1_mean = np.mean(knn_f1_list)
knn_acc_mean = np.mean(knn_acc_list)
knn_mcc_mean = np.mean(knn_mcc_list)
knn_auroc_mean = np.mean(knn_auroc_list)
knn_auprc_mean = np.mean(knn_auprc_list)

In [ ]:
# Calculate the confidence intervals for each metric.

metrics = {
    'Precision': knn_precision_list,
    'Recall': knn_recall_list,
    'F1 Score': knn_f1_list,
    'Accuracy': knn_acc_list,
    'MCC': knn_mcc_list,
    'AUROC': knn_auroc_list,
    'AUPRC': knn_auprc_list
}

result_strings = {}

for metric_name, metric_list in metrics.items():
    mean = round(np.mean(metric_list), 3)
    lower_ci, upper_ci = bootstrap_ci(metric_list)
    result_str = f"{metric_name}: {mean} ({lower_ci:.3f}, {upper_ci:.3f})"
    result_strings[metric_name] = result_str
    print(result_str)

knn_precision_str = result_strings['Precision']
knn_recall_str = result_strings['Recall']
knn_f1_str = result_strings['F1 Score']
knn_acc_str = result_strings['Accuracy']
knn_mcc_str = result_strings['MCC']
knn_auroc_str = result_strings['AUROC']
knn_auprc_str = result_strings['AUPRC']

In [ ]:
# Calculate ROC and PR curves.

knn_tpr_list = np.array(knn_tpr_list)
knn_mean_tprs = knn_tpr_list.mean(axis=0)
knn_std_tprs = knn_tpr_list.std(axis=0)

knn_prc_list = np.array(knn_prc_list)
knn_mean_prcs = knn_prc_list.mean(axis=0)
knn_std_prcs = knn_prc_list.std(axis=0)

# ROC, and PR Curves


In [ ]:
# Plot ROC curve.

f = pyplot.figure()
f.set_figwidth(12)
f.set_figheight(12)

pyplot.plot(xgb_base_fpr, xgb_mean_tprs, label='XGBoost AUROC: {:.3f}'.format(xgb_auroc_mean), color='r', linewidth = 3.5, alpha = 0.75)
pyplot.plot(lgb_base_fpr, lgb_mean_tprs, label='LightGBM AUROC: {:.3f}'.format(lgb_auroc_mean), color='g', linewidth = 3.5, alpha = 0.75)
pyplot.plot(svm_base_fpr, svm_mean_tprs, label='SVM AUROC: {:.3f}'.format(svm_auroc_mean), color='b', linewidth = 3.5, alpha = 0.75)
pyplot.plot(rf_base_fpr, rf_mean_tprs, label='Random Forest AUROC: {:.3f}'.format(rf_auroc_mean), color='c', linewidth = 3.5, alpha = 0.75)
pyplot.plot(knn_base_fpr, knn_mean_tprs, label='kNN AUROC: {:.3f}'.format(knn_auroc_mean), color='m', linewidth = 3.5, alpha = 0.75)

pyplot.plot([0, 1], [0, 1], linestyle = '--', linewidth=2)

pyplot.title('A', x = -0.075, y = 1.005, fontsize = 75, pad = 20)
pyplot.xlabel('False Positive Rate', fontsize = 22, fontweight = 'heavy', labelpad = 16)
pyplot.ylabel('True Positive Rate', fontsize = 22, fontweight = 'heavy', labelpad = 16)
pyplot.tick_params(axis="y",direction="out", labelsize = 16)
pyplot.tick_params(axis="x",direction="out", labelsize = 16)

leg = pyplot.legend(loc = 'lower right', fontsize = 18)

pyplot.savefig('/content/drive/MyDrive/HCMP-ML/roc.jpg', dpi=300)
pyplot.show()

In [ ]:
# Plot PR curve.

f = pyplot.figure()
f.set_figwidth(12)
f.set_figheight(12)

pyplot.plot(xgb_base_fpr, xgb_mean_prcs, label='XGBoost AUPRC: {:.3f}'.format(xgb_auprc_mean), color = 'r', linewidth = 3.5, alpha = 0.75)
pyplot.plot(lgb_base_fpr, lgb_mean_prcs, label='LightGBM AUPRC: {:.3f}'.format(lgb_auprc_mean), color = 'g', linewidth = 3.5, alpha = 0.75)
pyplot.plot(svm_base_fpr, svm_mean_prcs, label='SVM AUPRC: {:.3f}'.format(svm_auprc_mean), color = 'b', linewidth = 3.5, alpha = 0.75)
pyplot.plot(rf_base_fpr, rf_mean_prcs, label='Random Forest AUPRC: {:.3f}'.format(rf_auprc_mean), color = 'c', linewidth = 3.5, alpha = 0.75)
pyplot.plot(knn_base_fpr, knn_mean_prcs, label='kNN AUPRC: {:.3f}'.format(knn_auprc_mean), color = 'm', linewidth = 3.5, alpha = 0.75)

pyplot.title('B', x = -0.075, y = 1.005, fontsize = 75, pad = 20)
pyplot.xlabel('Recall', fontsize = 18, fontweight = 'heavy', labelpad = 16)
pyplot.ylabel('Precision', fontsize = 22, fontweight = 'heavy', labelpad = 16)
pyplot.tick_params(axis="y",direction="out", labelsize = 16)
pyplot.tick_params(axis="x",direction="out", labelsize = 16)

leg = pyplot.legend(loc = 'lower left', fontsize = 18)

pyplot.savefig('/content/drive/MyDrive/HCMP-ML/prc.jpg', dpi=300)
pyplot.show()

In [ ]:
roc = cv2.imread('/content/drive/MyDrive/HCMP-ML/roc.jpg')
prc = cv2.imread('/content/drive/MyDrive/HCMP-ML/prc.jpg')

fig1 = cv2.hconcat([roc, prc])

cv2_imshow(fig1)
cv2.imwrite('/content/drive/MyDrive/HCMP-ML/Figure 1.jpg', fig1, [cv2.IMWRITE_JPEG_QUALITY, 100])

# Results Summary

In [ ]:
# Summarize performance metrics in a table.

xgb_results = [xgb_precision_str.split(':')[1].strip(), xgb_recall_str.split(':')[1].strip(), xgb_f1_str.split(':')[1].strip(), xgb_acc_str.split(':')[1].strip(), xgb_mcc_str.split(':')[1].strip(), xgb_auroc_str.split(':')[1].strip(), xgb_auprc_str.split(':')[1].strip()]
lgb_results = [lgb_precision_str.split(':')[1].strip(), lgb_recall_str.split(':')[1].strip(), lgb_f1_str.split(':')[1].strip(), lgb_acc_str.split(':')[1].strip(), lgb_mcc_str.split(':')[1].strip(), lgb_auroc_str.split(':')[1].strip(), lgb_auprc_str.split(':')[1].strip()]
svm_results = [svm_precision_str.split(':')[1].strip(), svm_recall_str.split(':')[1].strip(), svm_f1_str.split(':')[1].strip(), svm_acc_str.split(':')[1].strip(), svm_mcc_str.split(':')[1].strip(), svm_auroc_str.split(':')[1].strip(), svm_auprc_str.split(':')[1].strip()]
rf_results = [rf_precision_str.split(':')[1].strip(), rf_recall_str.split(':')[1].strip(), rf_f1_str.split(':')[1].strip(), rf_acc_str.split(':')[1].strip(), rf_mcc_str.split(':')[1].strip(), rf_auroc_str.split(':')[1].strip(), rf_auprc_str.split(':')[1].strip()]
knn_results = [knn_precision_str.split(':')[1].strip(), knn_recall_str.split(':')[1].strip(), knn_f1_str.split(':')[1].strip(), knn_acc_str.split(':')[1].strip(), knn_mcc_str.split(':')[1].strip(), knn_auroc_str.split(':')[1].strip(), knn_auprc_str.split(':')[1].strip()]

results = pd.DataFrame({'XGBoost':xgb_results, 'LightGBM':lgb_results, 'SVM':svm_results, 'Random Forest':rf_results, 'kNN Results':knn_results})

results = results.T

results.columns = ['Precision', 'Recall', 'F1', 'Accuracy', 'MCC', 'AUROC', 'AUPRC']

results.to_csv('/content/drive/MyDrive/HCMP-ML/algorithm_performance.csv')

results

# SHAP plots

In [ ]:
import textwrap
def wrap_labels(ax, width, break_long_words=False):
    labels = []
    for label in ax.get_yticklabels():
        text = label.get_text()
        labels.append(textwrap.fill(text, width=width,
                                    break_long_words=break_long_words))
    ax.set_yticklabels(labels, rotation=0)

In [ ]:
#Calculate SHAP values for XGBoost.

xgb_explainer = shap.Explainer(xgb.predict, x)
xgb_shap_values = xgb_explainer(x)

In [ ]:
#Plot SHAP bar plot for XGBoost.

shap.plots.bar(xgb_shap_values, max_display = 15, show=False)

fig = pyplot.gcf()
ax = pyplot.gca()
fig.set_figheight(12)
fig.set_figwidth(5)

pyplot.title('A', x = -1.1, y = 1, fontsize = 50, pad = 20)
pyplot.xlabel("Mean |SHAP Value|", fontsize =12, fontweight = 'heavy', labelpad = 8)
pyplot.tick_params(axis="y",direction="out", labelsize = 12)
pyplot.tick_params(axis="x",direction="out", labelsize = 12)

wrap_labels(ax, 30)
ax.figure

pyplot.savefig('/content/drive/MyDrive/HCMP-ML/shap_xgb.jpg', dpi=300, bbox_inches='tight')

In [ ]:
#Calculate SHAP values for LightGBM.

lgb_explainer = shap.Explainer(lgb.predict, x)
lgb_shap_values = lgb_explainer(x)

In [ ]:
#Plot SHAP bar plot for LightGBM.

shap.plots.bar(lgb_shap_values, max_display = 15, show=False)

fig = pyplot.gcf()
ax = pyplot.gca()
fig.set_figheight(12)
fig.set_figwidth(5)

pyplot.title('B', x = -1.1, y = 1, fontsize = 50, pad = 20)
pyplot.xlabel("Mean |SHAP Value|", fontsize =12, fontweight = 'heavy', labelpad = 8)
pyplot.tick_params(axis="y",direction="out", labelsize = 12)
pyplot.tick_params(axis="x",direction="out", labelsize = 12)

wrap_labels(ax, 30)
ax.figure

pyplot.savefig('/content/drive/MyDrive/HCMP-ML/shap_lgb.jpg', dpi=300, bbox_inches='tight')

In [ ]:
#Calculate SHAP values for SVM.

svm_explainer = shap.Explainer(svm.predict, x)
svm_shap_values = svm_explainer(x)

In [ ]:
#Plot SHAP bar plot for SVM.

shap.plots.bar(svm_shap_values, max_display = 15, show=False)

fig = pyplot.gcf()
ax = pyplot.gca()
fig.set_figheight(12)
fig.set_figwidth(5)

pyplot.title('C', x = -1.1, y = 1, fontsize = 50, pad = 20)
pyplot.xlabel("Mean |SHAP Value|", fontsize =12, fontweight = 'heavy', labelpad = 8)
pyplot.tick_params(axis="y",direction="out", labelsize = 12)
pyplot.tick_params(axis="x",direction="out", labelsize = 12)

wrap_labels(ax, 30)
ax.figure

pyplot.savefig('/content/drive/MyDrive/HCMP-ML/shap_svm.jpg', dpi=300, bbox_inches='tight')

In [ ]:
#Calculate SHAP values for Random Forest.

rf_explainer = shap.Explainer(rf.predict, x)
rf_shap_values = rf_explainer(x)

In [ ]:
#Plot SHAP bar plot for Random Forest.

shap.plots.bar(rf_shap_values, max_display = 15, show=False)

fig = pyplot.gcf()
ax = pyplot.gca()
fig.set_figheight(12)
fig.set_figwidth(5)

pyplot.title('D', x = -1.1, y = 1, fontsize = 50, pad = 20)
pyplot.xlabel("Mean |SHAP Value|", fontsize =12, fontweight = 'heavy', labelpad = 8)
pyplot.tick_params(axis="y",direction="out", labelsize = 12)
pyplot.tick_params(axis="x",direction="out", labelsize = 12)

wrap_labels(ax, 30)
ax.figure

pyplot.savefig('/content/drive/MyDrive/HCMP-ML/shap_rf.jpg', dpi=300, bbox_inches='tight')

In [ ]:
#Calculate SHAP values for Random Forest.

knn_explainer = shap.Explainer(knn.predict, x)
knn_shap_values = knn_explainer(x)

In [ ]:
#Plot SHAP bar plot for Random Forest.

shap.plots.bar(knn_shap_values, max_display = 15, show=False)

fig = pyplot.gcf()
ax = pyplot.gca()
fig.set_figheight(12)
fig.set_figwidth(5)

pyplot.title('E', x = -1.1, y = 1, fontsize = 50, pad = 20)
pyplot.xlabel("Mean |SHAP Value|", fontsize =12, fontweight = 'heavy', labelpad = 8)
pyplot.tick_params(axis="y",direction="out", labelsize = 12)
pyplot.tick_params(axis="x",direction="out", labelsize = 12)

wrap_labels(ax, 30)
ax.figure

pyplot.savefig('/content/drive/MyDrive/HCMP-ML/shap_knn.jpg', dpi=300, bbox_inches='tight')

In [ ]:
shapA = cv2.imread('/content/drive/MyDrive/HCMP-ML/shap_xgb.jpg')
shapB = cv2.imread('/content/drive/MyDrive/HCMP-ML/shap_lgb.jpg')
shapC = cv2.imread('/content/drive/MyDrive/HCMP-ML/shap_svm.jpg')
shapD = cv2.imread('/content/drive/MyDrive/HCMP-ML/shap_rf.jpg')
shapE = cv2.imread('/content/drive/MyDrive/HCMP-ML/shap_knn.jpg')

shap1 = cv2.hconcat([shapA, shapB, shapC])
shap2 = cv2.hconcat([shapD, shapE])
cv2.imwrite('/content/drive/MyDrive/HCMP-ML/shap1.jpg', shap1, [cv2.IMWRITE_JPEG_QUALITY, 100])
cv2.imwrite('/content/drive/MyDrive/HCMP-ML/shap2.jpg', shap2, [cv2.IMWRITE_JPEG_QUALITY, 100])

shap1 = Image.open('/content/drive/MyDrive/HCMP-ML/shap1.jpg')
new_size = shap1.size
shap2 = Image.open('/content/drive/MyDrive/HCMP-ML/shap2.jpg')
old_size = shap2.size

new_im = Image.new("RGB", new_size, "White")
box = tuple((n - o) // 2 for n, o in zip(new_size, old_size))
new_im.paste(shap2, box)
new_im = new_im.save('/content/drive/MyDrive/HCMP-ML/shap2.jpg')

shap1 = cv2.imread('/content/drive/MyDrive/HCMP-ML/shap1.jpg')
shap2 = cv2.imread('/content/drive/MyDrive/HCMP-ML/shap2.jpg')

fig2 = cv2.vconcat([shap1, shap2])

cv2_imshow(fig2)
cv2.imwrite('/content/drive/MyDrive/HCMP-ML/Figure 2.jpg', fig2, [cv2.IMWRITE_JPEG_QUALITY, 100])